<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import numpy as np 
import pandas as pd
import os, io
import datetime as dt
import pandas_datareader.data as web
import requests

# **TODO**


> Vergleiche mehrere Aktien miteinander



> Sende Push zu der Aktie, mit der besten Bewertung



> Prüfen wie stark der CCI gewachsen ist in den letzten 14 Tagen

# **Weitere Ideen**


> Aktien Sektoren zuordnen und den Bot nach Sektoren abfragen. Der Bot soll dann die stärkste Aktie aus dem Sektor zurückgeben. Ziemlich cool, wenn man weiß das zum Beispiel der Wasserstoffsektor gerade extrem am steigen ist.




> Sobald man sich für eine Aktie entschieden, wäre es ja auch interessant zu wissen, wann man am besten aussteigen sollte. Dann könnte man Abfragen schreiben und über den Bot eine Push rausschicken, ob man eine bestimmte Aktie, die man mit dem Bot teilt, verkaufen soll oder nicht, weil sich gerade der Kurs schlecht entwickelt.





# **Indikatoren**


> **MACD & Signallinie**: Der MACD (Moving Average Convergence/Divergence) wird in der Chartanalyse genutzt um Trends sowie Kauf-und Verkaufssignale zu erkennen. Dabei werden zwei exponentiell gewichtete gleitende Durchschnitte (26 und 12 Tage) berechnet und voneinander subtrahiert. Des Weiteren wird eine Signallinie berechnet. Hierbei handelt es sich um den 9 Tage exponentiell geglätteten Durchschnitt. Ein steigender MACD ist ein Indikator für einen Aufwärtstrend, ein sinkender MACD ein Indikator für einen Abwärtstrend. Schneidet der MACD die Signallinie von unten nach oben, so liegt ein Kaufsignal vor. Wird die Signallinie vom MACD von oben nach unten geschnitten, so liegt ein Verkaufssignal vor.



> **RSI**: Der RSI (Relative-Stärke-Index) ist ein Marktindikator. Er wird berechnet, indem der gleitende Mittelwert der Auf-und Abwärtsveränderungen für einen bestimmten Zeitraum (in der Regel 14 Tage) ermittelt wird. Der RSI kann Werte zwischen 0 und 100 einnehmen. Die Werte 30 und 70 spielen beim RSI eine besondere Rolle: Bei einem Wert unter 30 gilt der Wert als überverkauft. Dies bedeutet, dass eine Kurserholung (Steigerung) in nächster Zeit anstehen könnte. Bei einem Wert über 70 gilt die Aktie als überkauft. Dies ist ein Indiz dafür, dass der Kurs in nächster Zeit fallen könnte.




> **Bollinger Band**: Es existieren eigentlich zwei Bollinger Bänder, die gemeinsam mit dem gleitenden Mittel, dem arithmetischen Mittel der Kurse, dann das sogenannte Bollinger Band bilden. Mit Hilfe des Bollinger Bands werden der Verlauf der letzten Tage eines Wertpapieres und dessen Trend visualisiert. Man sieht, wo der Kurs durchschnittlich lag und wie stark er an welchen Tagen vom Mittelwert abwich. In der Regel verwendet man dafür den Zeitraum der letzten 20 Tage.





> **Bollinger Bands oder Keltner Channels?** Schau hier warum die BB die bessere Wahl wären: https://tradeciety.com/keltner-channel-vs-bollinger-bands-one-better/ Aber statt der Verwendung der Standardabweichung zur Darstellung der Bollinger Bänder wird die Average True Range von Wilders eingesetzt. Dieser Unterschied bewirkt, dass der Keltner Channel ruhiger und gleichmäßiger aussieht als die Bollinger Bänder.



> **EMA**: Exponential Moving Average verwendet man, um Kursschwankungen zu glätten und den zugrunde liegenden Trend dadurch besser zu erkennen. Hierzu betrachtet man einen Kursverlauf für einen bestimmten Zeitraum in der Vergangenheit und berechnet daraus den Durchschnitt. **Formel**: https://admiralmarkets.com/de/wissen/articles/forex-indicators/ema-der-exponentiell-gleitende-durchschnitt-als-universelles-hilfsmittel-fur-ihr-trading




> **Commodity Channel Index (CCI)**: Für den CCI wird die Abweichung zwischen Kurs und dessen Gleitenden Durchschnitt herangezogen, um das Momentum zu erfassen. Um dies zu bewerkstelligen, wird die Abweichung zwischen dem aktuellen Betrachtungszeitraum und der durchschnittlichen Abweichung der vergangenen X Zeitperioden verglichen. Im Gegensatz zum Stochastik-Oszillator und dem RSI sind die Werte des CCI nicht begrenzt. Aber in der Praxis bewegt sich der CCI aufgrund seiner Konstruktion zwischen -100 und +100. 
Bemerkenswerterweise werden beim CCI nicht die Schlusskurse, sondern die typischen Kurse (Hoch + Tief + Schlusskurs / 3) zugrunde gelegt. Dann wird die Abweichung bzw. Entfernung zwischen typischem Kurs und Gleitendem Durchschnitt ermittelt. **Formel**: https://www.kagels-trading.de/technische-indikatoren-trader/ **WICHTIG**: Wie der Name schon sagt, gründet der CCI auf der Annahme, dass sich die Kurse normalerweise innerhalb eines unsichtbaren Kanals um den Gleitenden Durchschnitt herum aufhalten.














# **Indikatoren mit denen wir anfangen sollten**


> 1. EMA


> 2. MACD


> 3. Bollinger Bänder



> 4. RSI












In [83]:
#indicator = pd.DataFrame(df, index=df.index)

In [84]:
def calc_indicators(df):
  ema_25 = 25
  ema_100 = 100
  sma_25 = 25
  sma_100 = 100
  ema_short = 12
  ema_long = 26
  bollinger_avg = 20
  signalline_span = 9

  df['ema_25'] = df['Adj Close'].ewm(span=ema_25, adjust=False).mean()
  df['ema_100'] = df['Adj Close'].ewm(span=ema_100, adjust=False).mean()

  df['sma_25'] = df['Adj Close'].rolling(window=sma_25).mean()
  df['sma_100'] = df['Adj Close'].rolling(window=sma_100).mean()

  df['short_avg_ma'] = df['Adj Close'].rolling(window=bollinger_avg).mean()
  df['short_avg_std'] = df['Adj Close'].rolling(window=bollinger_avg).std()

  df['bollinger'] = df['Adj Close'].ewm(span=bollinger_avg, adjust=False).mean()
  df['bollinger_lower_band'] = df['short_avg_ma'] - (df['short_avg_std'] *2)
  df['bollinger_upper_band'] = df['short_avg_ma'] + (df['short_avg_std'] *2)

  df['ema_short'] = df['Adj Close'].ewm(span=ema_short, adjust=False).mean()
  df['ema_long'] = df['Adj Close'].ewm(span=ema_long, adjust=False).mean()
  df['macd'] = df['ema_short'] - df['ema_long']
  df['signalline'] = df['macd'].ewm(span=signalline_span, adjust=False).mean()

  df['RSI'] = calc_rsi(df)

  df['CCI'] = calc_cci(df)

  # df['OBV_MA'] = calc_obv(df)
   
  df = calculate_dead_cat_bounce_v2(df)

  return df

In [85]:
def calc_rsi(df, n=14):
    delta = df['Adj Close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    # RolUp = pd.rolling_mean(dUp, n)
    # RolDown = pd.rolling_mean(dDown, n).abs()

    # Calculate the EWMA
    RolUp = dUp.ewm(span=n).mean()
    RolDown = dDown.abs().ewm(span=n).mean()

    # Calculate the RSI based on EWMA
    RS = RolUp / RolDown
    RSI = 100 - (100/(1+RS))

    # Calculate the SMA
    roll_up2 = dUp.rolling(n).mean()
    roll_down2 = dDown.abs().rolling(n).mean()

    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))

    own_RSI=(RSI+RSI2)/2
    return own_RSI

In [86]:
def calc_cci(df, n=14, constant=0.015):
    tippingPrice = (df['High'] + df['Low'] + df['Adj Close']) / 3
    tippingPrice_mean = tippingPrice.rolling(n).mean()
    tippingPrice_std = tippingPrice.rolling(n).std()
    
    CCI = pd.Series((tippingPrice - tippingPrice_mean) / (constant * tippingPrice_std)) 
    return CCI

In [87]:
### To Be Adjusted

def calc_obv(df, n = 20):
    i = 0
    OBV = [0]

    while i < df.index[-1]:
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] > 0:
            OBV.append(df.loc[i + 1, 'Volume'])
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] == 0:
            OBV.append(0)
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] < 0:
            OBV.append(-df.loc[i + 1, 'Volume'])
        i = i + 1
    OBV = pd.Series(OBV)
    OBV_ma = pd.Series(OBV.rolling(n, min_periods=n).mean())

    return OBV_ma

In [88]:
def calculate_dead_cat_bounce_v2(df):
  set_freq = '2D'

  df['body_mid'] = (df['Adj Close'] + df['Open'])/2

  pct_change_close = df['body_mid'].pct_change(freq=set_freq, fill_method='ffill')
  pct_change_close.dropna(inplace=True)

  dead_cat_bounce = np.where((df.signalline > df.macd) &
                            ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                            ((df.Open < df.bollinger_lower_band)|(df.Close < df.bollinger_lower_band)),
                            True, False)

  df['dead_cat_bounce'] = dead_cat_bounce
  del df['body_mid']

  return df

In [89]:
def combine_stocks_to_dataframe(id_list):
  start = dt.datetime(2020,1,1)
  end = dt.datetime.now()
  df_list = []

  for id in id_list:
    stock = web.DataReader(id, 'yahoo', start, end) # load data
    stock['stock_id'] = id
    stock = calc_indicators(stock) # calc indicators for every stock
    stock.dropna(inplace=True) # maybe delete this later
    df_list.append(stock)

  df_complete = pd.concat(df_list)
  
  return df_complete

In [90]:
df_complete = combine_stocks_to_dataframe(['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE', 'PLUG', 'MDO.DE', 'PYPL', 'SQ', 'V', 'BNTX', 'CVAC', 'YNDX', 'SPLK', 'SYK', 'SHL.DE', 'PHG', 'ISRG'])

In [91]:
df_yd = df_complete.drop_duplicates(subset='stock_id', keep='last')

In [92]:
# def compute_score(row):
#   score=0
#   if row['ema_25']>row['ema_100']:
#     score+=1
#     if row['macd']>row['signalline']:
#       score+=1
#       if row['Adj Close']>row['bollinger_upper_band']:
#         return score
#       else:
#         if ((row['RSI']>=50) & (row['RSI']<=70)) | (row['RSI'] <= 30):
#           score+=1
#         if row['Adj Close']<row['bollinger_lower_band']:
#           score+=1
#   return score

In [93]:
def compute_score_v2(row):
  score=0
  if row['ema_25']>row['ema_100']:
    score+=1
  if row['macd']>row['signalline']:
    score+=1
  if row['Adj Close']>row['bollinger_upper_band']:
    score-=1
  if ((row['RSI']>=50) & (row['RSI']<=70)) | (row['RSI'] <= 30):
    score+=1
  if row['Adj Close']<row['bollinger_lower_band']:
    score+=1
  if row['CCI']<-100:
      score+=1
  if row['Adj Close']>row['sma_25']:
      score+=1
  return score

In [97]:
def ema_score(row):
    ema_25 = row['ema_25']
    ema_100 = row['ema_100']
    score = (ema_25/ema_100)-1
    # if score < 0:
    #     score=0
    return score

def macd_signalline_score(row): # need to change something here
    # relative change might be distorted if values near by 0
    macd = row['macd']
    signalline = row['signalline']
    score = (macd/signalline)-1
    # if score < 0:
    #     score=0
    return score

def bollinger_score(row):
    lower_band = row['bollinger_lower_band']
    upper_band = row['bollinger_upper_band']
    middle_band = row['bollinger']
    adj_close = row['Adj Close']
    if (adj_close>middle_band):
        if adj_close>upper_band:
            score = ((adj_close/upper_band)-1)*(-1)
        else:
            score = ((adj_close/((upper_band-middle_band)*0.5+middle_band))-1)*(-1)
    else:
        if (adj_close<lower_band):
            score = ((adj_close/lower_band)-1)*(-1)
        else:
            score = (adj_close/middle_band)-1
    return score

def sma_score(row):
    sma_25 = row['sma_25']
    adj_close = row['Adj Close']
    score = (adj_close/sma_25)-1
    return score

In [102]:
def relative_score(row):
    r = row
    ema_scr = ema_score(r)
    macd_signalline_scr = macd_signalline_score(r)
    bollinger_scr = bollinger_score(r)
    sma_scr = sma_score(r)
    score = ema_scr+macd_signalline_scr+bollinger_scr+sma_scr
    return score

In [109]:
-0.6/1

-0.6

In [104]:
df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)
df_yd

<ipython-input-104-498953b33bc0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)


,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,...,bollinger_lower_band,bollinger_upper_band,ema_short,ema_long,macd,signalline,RSI,CCI,dead_cat_bounce,score
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-07,3326.290039,3291.030029,3319.090088,3319.849121,2361809.0,3319.849121,AMZN,3324.666230,3224.839730,3353.191162,...,3258.457391,3484.376520,3350.132011,3322.093958,28.038052,48.185184,44.534081,-59.293904,False,-0.402385
2021-05-07,131.070007,129.475006,130.850006,130.824997,35201303.0,130.824997,AAPL,130.522521,126.816380,131.765001,...,128.286427,136.789074,131.079012,130.454225,0.624787,1.378865,43.822818,-55.424378,False,-0.525053
2021-05-07,142.729996,141.839996,141.919998,142.240005,166476.0,142.240005,SAP,137.791493,132.213030,138.400399,...,133.480790,146.819209,140.406660,137.589075,2.817585,3.430751,57.524168,24.367323,False,-0.104626
2021-05-07,294.079987,292.000092,292.200012,292.760010,416737.0,292.760010,ACN,286.405647,266.831591,287.925983,...,283.891585,294.614701,290.358001,286.062074,4.295926,5.036604,63.936887,117.814447,False,-0.061745
2021-05-07,265.799988,258.200012,265.399994,260.399994,73759.0,260.399994,VOW.DE,273.111529,233.961265,282.720002,...,252.826611,301.473393,268.876563,272.981979,-4.105416,-0.303948,26.564154,-76.339096,False,12.549546
2021-05-07,23.930000,22.590000,22.980000,23.230000,10999081.0,23.230000,PLUG,28.639294,35.314564,28.103200,...,22.661476,31.097524,25.956241,28.859436,-2.903196,-2.927525,35.557790,-103.158000,False,-0.527762
2021-05-07,194.850006,193.050003,194.550003,193.949997,1768.0,193.949997,MDO.DE,192.759483,184.346239,193.858000,...,191.868260,196.051740,194.170868,192.610850,1.560018,2.010435,50.613634,-6.665829,False,-0.173838
2021-05-07,258.640015,252.460098,255.919998,256.526489,4516274.0,256.526489,PYPL,259.387925,247.400897,263.164659,...,248.759175,280.044474,258.692976,259.330955,-0.637979,1.640560,42.395126,-55.648910,False,-1.377311
2021-05-07,241.699402,233.429993,240.089996,237.600006,7254525.0,237.600006,SQ,242.085755,229.442515,247.994399,...,224.343811,272.483189,240.188485,242.067304,-1.878819,1.059233,44.598441,-50.831983,False,-2.778634


In [22]:
# MACD, RSI, EMA, SMA
# WHAT TO INCLUDE: OBV

def compute_trend_indicators(row):
    score = 0


In [241]:
#print(df_yd.apply(lambda row: compute_score(row), axis=1).sort_values())

# df_complete = combine_stocks_to_dataframe(['AMZN', 'V', 'AAPL', 'VOW.DE', 'PLUG'])

In [238]:
df_yd['score'] = df_yd.apply(lambda row: compute_score_v2(row), axis=1)
df_yd

<ipython-input-238-a76c1241de50>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yd['score'] = df_yd.apply(lambda row: compute_score_v2(row), axis=1)


,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,...,bollinger_lower_band,bollinger_upper_band,ema_short,ema_long,macd,signalline,RSI,CCI,dead_cat_bounce,score
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-29,3514.449951,3435.000000,3505.100098,3471.310059,7557500.0,3471.310059,AMZN,3316.873990,3209.891694,3290.065195,...,3188.322603,3504.501371,3382.812982,3313.028760,69.784222,62.696133,66.642802,134.010947,False,4
2021-04-29,137.070007,132.449997,136.470001,133.479996,150777100.0,133.479996,AAPL,130.832513,126.389898,129.674000,...,124.966574,138.694427,132.991247,130.707961,2.283286,2.339229,53.607451,50.476640,False,3
2021-04-29,144.190002,142.470001,144.070007,143.779999,876500.0,143.779999,SAP,136.389470,131.222463,134.279598,...,125.729486,148.872510,140.756615,136.138273,4.618342,4.188644,77.736819,61.484521,False,3
2021-04-29,293.279999,290.600006,292.079987,292.540009,1247800.0,292.540009,ACN,283.414402,263.723115,284.316062,...,278.051114,294.793964,288.683981,283.014309,5.669672,6.071949,70.784425,97.974152,False,2
2021-04-30,268.200012,259.799988,265.000000,266.600006,49942.0,266.600006,VOW.DE,281.036513,227.509074,293.360002,...,267.334733,318.325274,284.840341,280.303856,4.536485,10.348830,24.830131,-149.051983,False,4
2021-04-29,30.340000,27.980000,30.049999,28.270000,35893500.0,28.270000,PLUG,31.057725,36.641739,30.272400,...,23.368395,35.433604,28.538968,31.285309,-2.746341,-3.453707,41.872486,52.793222,False,1
2021-04-30,195.100006,193.550003,193.550003,194.350006,409.0,194.350006,MDO.DE,191.142023,182.398320,192.527200,...,190.467259,196.054743,193.537612,190.934056,2.603556,3.112486,54.270379,58.012476,False,3
2021-04-29,273.869995,263.529999,273.260010,267.850006,5657900.0,267.850006,PYPL,262.547112,246.496365,259.579599,...,249.821348,280.095651,266.784681,262.298547,4.486134,4.448112,52.418188,-4.166125,False,4
2021-04-29,259.880005,245.009995,257.358002,247.759995,6567800.0,247.759995,SQ,246.500220,228.687459,243.415199,...,228.140437,273.484560,250.218750,246.246285,3.972465,4.660090,43.314498,-31.673402,False,2


In [240]:
df_complete[df_complete.dead_cat_bounce==True]

,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,...,bollinger,bollinger_lower_band,bollinger_upper_band,ema_short,ema_long,macd,signalline,RSI,CCI,dead_cat_bounce
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-28,110.339996,107.699997,109.669998,108.110001,5486700.0,108.110001,SAP,145.454121,148.740207,149.296800,...,143.028637,116.717681,179.038320,136.493572,145.849917,-9.356345,-4.462065,9.557939,-137.607496,True
2020-10-29,109.870003,106.059998,106.800003,108.629997,2824800.0,108.629997,SAP,142.621496,147.945945,147.508400,...,139.752576,110.056879,180.880120,132.206868,143.092886,-10.886018,-5.746856,9.844080,-112.666718,True
2020-09-24,220.449997,213.149994,218.869995,214.419998,4828600.0,212.199539,ACN,231.450523,217.639759,234.501379,...,231.527577,221.427159,247.008135,230.264042,231.385243,-1.121200,1.208118,24.030213,-209.301065,True
2020-09-25,215.070007,210.419998,213.029999,214.630005,3691100.0,212.407364,ACN,229.985664,217.536147,233.631283,...,229.706604,217.013155,248.825311,227.516861,229.979474,-2.462613,0.473972,25.204056,-159.812921,True
2020-07-31,140.199997,132.699997,138.000000,132.699997,91147.0,128.430832,VOW.DE,140.943584,141.077473,141.415214,...,140.850220,133.049339,150.640522,139.987165,140.940759,-0.953594,0.590261,22.148804,-173.978706,True
2020-10-28,136.800003,131.800003,136.300003,133.500000,142287.0,133.500000,VOW.DE,144.791133,144.375611,145.078564,...,144.445724,138.177402,152.812600,143.312620,144.841505,-1.528885,-0.338882,23.690957,-193.654246,True
2021-02-18,54.480000,50.060001,51.880001,50.230000,34222100.0,50.230000,PLUG,59.380188,39.722116,64.281600,...,60.514456,54.706293,73.482707,60.716823,59.106783,1.610041,4.333278,21.347698,-171.602557,True
2021-03-25,34.180000,30.100000,31.000000,34.099998,35469800.0,34.099998,PLUG,43.305191,40.869021,43.284000,...,41.900997,31.200699,52.552301,39.032677,43.541913,-4.509236,-3.954665,36.760703,-107.943324,True
2021-04-15,28.770000,24.610001,28.750000,26.090000,69174300.0,26.090000,PLUG,35.218989,38.667813,35.596400,...,33.773345,26.706104,39.896896,31.315878,35.491190,-4.175311,-4.009442,26.838968,-140.075752,True


In [243]:
def bot_warn_me(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = '1790740607:AAE7Omfuea3tG3iii9A1qX-vQJ9pdLFcO0s'
    bot_chat_id = '908941210'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()

In [245]:
dead_cat_bounce = calculate_dead_cat_bounce()
#bot_warn_me('Watch Out! Check Code')
dead_cat_bounce

NameError: name 'calculate_dead_cat_bounce' is not defined